In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
historical_data = pd.read_csv("historical_fuel_requirements.csv")
customer_assignments = pd.read_csv("historical_fuel_requirements_customer_assignment.csv")

In [ ]:
# Function to train SARIMA model and make predictions for next week
def train_and_predict_sarima(train_data, test_data):
    # Train SARIMA model
    model = SARIMAX(train_data['Requirement'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 52))
    sarima_model = model.fit()
    # Forecast fuel requirement for the next week
    forecast = sarima_model.forecast(steps=1)
    return forecast[0]

predicted_fuel_requirements = pd.DataFrame(columns=['Customer_ID', 'Date', 'Fuel_Type', 'Predicted_Requirement'])

In [ ]:
# Iterating over each week and corresponding customer assignments
for week_start_date in historical_data['Date'].unique():
    # Filtering customer assignments for the current week
    week_assignments = customer_assignments[customer_assignments['Date'] == week_start_date]
    if len(week_assignments) == 0:
        continue  # Skipping if no assignments found for this week
    for index, row in week_assignments.iterrows():
        customer_id = row['Customer_ID']  
        customer_data = historical_data[(historical_data['Date'] == week_start_date) & 
                                        (historical_data['Customer_Assignment'] == customer_id)]
        for fuel_type in ['HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 
                          'LSHS_Requirement', 'SKO_Requirement', 'MS_Requirement']:
            train_data = customer_data[customer_data['Date'] < week_start_date]
            test_data = customer_data[customer_data['Date'] == week_start_date]
            # Training SARIMA model and make predictions for next week
            predicted_requirement = train_and_predict_sarima(train_data, test_data)
            predicted_fuel_requirements = predicted_fuel_requirements.append({'Customer_ID': customer_id,
                                                                              'Date': week_start_date,  
                                                                              'Fuel_Type': fuel_type,
                                                                              'Predicted_Requirement': predicted_requirement},
                                                                             ignore_index=True)

In [ ]:
predicted_fuel_requirements.to_csv("Time_Series_Fuel_Requirement_Prediction_Data.csv", index=False)